# Handling overscan, trimming, and bias subtraction


The bias in a CCD camera is a DC offset applied to all pixels so that when the voltage in each pixel is converted to a number the number will always be positive. In an ideal CCD the bias would be the same for every pixel and not change over time. In practice, the bias is slightly different for each pixel, and can vary by a count or two from night to night or during a night.

A bias *image* is a picture taken with the shutter close and zero exposure time; think about it as a command to the camera to do whatever it usually does to prepare the camera's electronics to take an image and then immediately read out the CCD as though you had taken a picture.

The discussion of bias assumes you have taken images with an overscan region. If your images do not have overscan, simply skip those steps.

The progression here is to "calibrate" the bias images by subtracting overscan (again, simply skip that if your images do not have overscan), trim the overscan from the bias images and combine all of the bias images to make a "master" bias. 

In [ ]:
from astropy.nddata import CCDData
from astropy.visualization import hist

import matplotlib.pyplot as plt

from convenience_functions import show_image

download_base_url = 'http://physics.mnstate.edu/craig/ccd-guide/'

## A sample bias image

The image below is a single bias frame from an [Andor Apogee Aspen CG16M](http://www.andor.com/pdfs/specifications/Apogee_Aspen_CG16M_Specifications.pdf), a low-end 4k × 4k CCD with a [Kodak KAF-16803 sensor chip](http://www.onsemi.com/pub/Collateral/KAF-16803-D.PDF). That model camera has a typical bias level around 1000 and read noise around 10 e$^-$, though the precise value from camera to camera.

In [ ]:
one_bias = CCDData.read(download_base_url + 'dark-test-0100bias.fit', unit='adu')

In [ ]:
show_image(one_bias, cmap='gray')

Note a few things:

+ The bias level in this specific camera is about 1023 (the mid-range of the colorbar).
+ The image is brighter on the left and right edges. This "amplifier glow" is frequently present and caused by the CCD electronics.
+ There are several vertical lines; these are columns for which the bias level is consistently higher.
+ There is noticeable "static" in the images; that is read noise.
+ None of the variations are particularly large. 

In [ ]:
hist(one_bias.data.flatten(), bins=800)

plt.grid()
plt.xlim(975, 1150)

In [ ]:
master = CCDData.read(download_base_url + 'master_bias.fit', unit='adu')

### 100 bias images combined

In [ ]:
show_image(master, cmap='gray')

## Cross-section 

Discuss edges, sensor glow, overscan.

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(one_bias.data[1000, :], label='single image')
plt.plot(master.data[1000, :], label='100 combined')
plt.grid()
plt.legend()
#plt.xlim(4000, 4110)

## Pixel histogram

More combined means narrower distribution (less noise)

In [ ]:
hist(one_bias.data.flatten(), bins=800, alpha=0.5, label='One bias image')
hist(master.data.flatten(), bins=800, alpha=0.5, label='100 bias images combined')
plt.semilogy()
plt.grid()
plt.legend()
plt.xlim(900, 1400)

### Histogram excluding left/right edges of the sensor

In [ ]:
hist(one_bias.data[:, 5:-15].flatten(), bins=4000, alpha=0.5, label='One bias image')
hist(master.data[:, 5:-15].flatten(), bins=4000, alpha=0.5, label='100 bias images combined')
#plt.semilogy()
plt.grid()
plt.legend()
plt.xlim(980, 1060)